<a href="https://colab.research.google.com/github/OscarJCC/Professional-Internship-Project-Machine-Learning/blob/main/Entrenamiento_pe12_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import json
import os
from tensorflow import keras
from glob import glob
from tqdm import notebook
from sklearn import preprocessing
from keras import Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, BatchNormalization
from keras import optimizers
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import sys, os
from sklearn.model_selection import train_test_split
from keras.models import load_model
from keras.models import model_from_json
from keras.utils import to_categorical
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split

In [ ]:
!pip install -U git+https://github.com/qubvel/efficientnet
from efficientnet.keras import EfficientNetB7
from efficientnet.keras import preprocess_input

  Cloning https://github.com/qubvel/efficientnet to /tmp/pip-req-build-zllihv59
  Running command git clone --filter=blob:none --quiet https://github.com/qubvel/efficientnet /tmp/pip-req-build-zllihv59
  Resolved https://github.com/qubvel/efficientnet to commit f7f3e736c113b872caf53dae9fbbda996a8eb87d
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.4 MB/s eta 0:00:00
  Created wheel for efficientnet: filename=efficientnet-1.1.1-py3-none-any.whl size=18427 sha256=e079443b671c1a0db05684c1c70f979dbb3cbbc1b09d18a80b97e66712956e53
  Stored in directory: /tmp/pip-ephem-wheel-cache-o72_dewt/wheels/bb/0f/38/154505705a0899276d49f521ee621a61bc2a8767026220f751
Successfully built efficientnet


In [ ]:
eff_net = EfficientNetB7(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

258434480/258434480 [==============================] - 3s 0us/step


In [ ]:
x = eff_net.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(7, activation='softmax')(x) # Colocar el numero de carpetas con imagenes a analizar
model = Model(inputs=eff_net.input, outputs=predictions)

# fix the feature extraction part of the model
# fix the feature extraction part of the model
for layer in eff_net.layers:
    layer.trainable = True

model.compile(optimizers.legacy.RMSprop(lr=0.0001, decay=0.0001/50),loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 stem_conv (Conv2D)          (None, 112, 112, 64)         1728      ['input_1[0][0]']             
                                                                                                  
 stem_bn (BatchNormalizatio  (None, 112, 112, 64)         256       ['stem_conv[0][0]']           
 n)                                                                                               
                                                                                                  
 stem_activation (Activatio  (None, 112, 112, 64)         0         ['stem_bn[0][0]']         

/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/rmsprop.py:144: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


 block1c_bn (BatchNormaliza  (None, 112, 112, 32)         128       ['block1c_dwconv[0][0]']      
 tion)                                                                                            
                                                                                                  
 block1c_activation (Activa  (None, 112, 112, 32)         0         ['block1c_bn[0][0]']          
 tion)                                                                                            
                                                                                                  
 block1c_se_squeeze (Global  (None, 32)                   0         ['block1c_activation[0][0]']  
 AveragePooling2D)                                                                                
                                                                                                  
 block1c_se_reshape (Reshap  (None, 1, 1, 32)             0         ['block1c_se_squeeze[0][0]']  
 e)       

In [ ]:
people_x_train = np.load('/content/drive/My Drive/KDEF_and_AKDEF_R/NrmEfficientNet/People_Data/People_x_train_pe12.npy')
people_x_valid = np.load('/content/drive/My Drive/KDEF_and_AKDEF_R/NrmEfficientNet/People_Data/People_x_valid.npy')
people_y_train = np.load('/content/drive/My Drive/KDEF_and_AKDEF_R/NrmEfficientNet/People_Data/People_y_train_pe12.npy')
people_y_valid = np.load('/content/drive/My Drive/KDEF_and_AKDEF_R/NrmEfficientNet/People_Data/People_y_valid.npy')

Extraccion de las imagenes de cada persona colocandolas en train y valid

In [ ]:
x_train, x_valid, y_train, y_valid = [], [], [], []

for i in range(people_x_train.shape[0]):
  for j in range(7):
    x_train.append(people_x_train[i][j])
    y_train.append(people_y_train[i][j])

for i in range(people_x_valid.shape[0]):
  for j in range(7):
    x_valid.append(people_x_valid[i][j])
    y_valid.append(people_y_valid[i][j])

x_train, y_train, x_valid, y_valid = map(np.asarray, [x_train, y_train, x_valid, y_valid])

print(f'X_train shape:{x_train.shape}\nY_train shape:{y_train.shape}\nX_valid shape:{x_valid.shape}\nY_valid shape:{y_valid.shape}')

X_train shape:(427, 224, 224, 3)
Y_train shape:(427, 7)
X_valid shape:(49, 224, 224, 3)
Y_valid shape:(49, 7)


Desorganizando imagenes aleatoriamente de train y valid respetando la asosciacion de las imagenes en 'x' y 'y'

In [ ]:
xy_train = list(zip(x_train,y_train))
xy_valid = list(zip(x_valid,y_valid))

random.seed(42)
random.shuffle(xy_train)
random.shuffle(xy_valid)

x_train, y_train = zip(*xy_train)
x_valid, y_valid = zip(*xy_valid)

x_train, y_train, x_valid, y_valid = map(np.asarray, [x_train, y_train, x_valid, y_valid])

In [ ]:
# En weights colocar archivo generando en el test de simulacion de aprendizaje
nb_epoch = 20 #Numero de etapas
learning_rate = 0.0001
batch_size = 10 # Tamaño de lote
weights="/content/drive/My Drive/KDEF_and_AKDEF_R/NrmEfficientNet/efficientnet-b7_weights_tf_dim_ordering_tf_kernels_autoaugment_notop.h5"
callbacks = [ModelCheckpoint(weights, monitor='val_loss', save_best_only=True, verbose=0)]

In [ ]:
%%time
# Train model
history = model.fit(x_train , y_train, batch_size=batch_size,epochs=nb_epoch,validation_data=(x_valid, y_valid),verbose=1, callbacks=callbacks, use_multiprocessing=True)

Epoch 1/20
43/43 [==============================] - ETA: 0s - loss: 1.3229 - accuracy: 0.5082

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


43/43 [==============================] - 109s 1s/step - loss: 1.3229 - accuracy: 0.5082 - val_loss: 1.9835 - val_accuracy: 0.1429
Epoch 2/20
43/43 [==============================] - 23s 542ms/step - loss: 0.5146 - accuracy: 0.8314 - val_loss: 2.0222 - val_accuracy: 0.1429
Epoch 3/20
43/43 [==============================] - 23s 539ms/step - loss: 0.2590 - accuracy: 0.9110 - val_loss: 2.0310 - val_accuracy: 0.1429
Epoch 4/20
43/43 [==============================] - 24s 548ms/step - loss: 0.2580 - accuracy: 0.9180 - val_loss: 2.0115 - val_accuracy: 0.1429
Epoch 5/20
43/43 [==============================] - 24s 547ms/step - loss: 0.2032 - accuracy: 0.9368 - val_loss: 2.0167 - val_accuracy: 0.1429
Epoch 6/20
43/43 [==============================] - 23s 544ms/step - loss: 0.1178 - accuracy: 0.9625 - val_loss: 2.0206 - val_accuracy: 0.1224
Epoch 7/20
43/43 [==============================] - 23s 545ms/step - loss: 0.1139 - accuracy: 0.9625 - val_loss: 2.0526 - val_accuracy: 0.2245
Epoch 8/20
4

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

print('Training acc:{}'.format(max(acc)))
print('Validation acc:{}'.format(max(val_acc)))

print('Training loss:{}'.format(max(loss)))
print('Vallidation loss:{}'.format(max(val_loss)))

Training acc:0.9953161478042603
Validation acc:0.795918345451355
Training loss:1.3228840827941895
Vallidation loss:2.0634257793426514


In [ ]:
fer_json = model.to_json()
with open("/content/drive/My Drive/KDEF_and_AKDEF_R/NrmEfficientNet/Result_Predict/pe12_Data/FER_pe12.json", "w") as json_file:
  json_file.write(fer_json)
model.save_weights("/content/drive/My Drive/KDEF_and_AKDEF_R/NrmEfficientNet/Result_Predict/pe12_Data/FER_pe12.h5")
print("Saved Model to Disk")

Saved Model to Disk


Extraccion De Los Datos De Las Personas A Eliminar

In [ ]:
x_valid_pe1 = np.load('/content/drive/My Drive/KDEF_and_AKDEF_R/NrmEfficientNet/People_Data/x_valid_pe1.npy')
x_valid_pe2 = np.load('/content/drive/My Drive/KDEF_and_AKDEF_R/NrmEfficientNet/People_Data/x_valid_pe2.npy')

Guardado De Resultados y Predicciones

In [ ]:
result = np.asarray([max(acc),max(val_acc),max(loss),max(val_loss)])
predict_pe1 = np.asarray(model.predict(x_valid_pe1[0]))
predict_pe2 = np.asarray(model.predict(x_valid_pe2[0]))

print("Result:\n",result)
print("Predict_pe1:\n",predict_pe1)
print("Predict_pe2:\n",predict_pe2)

1/1 [==============================] - 0s 65ms/step
Result:
 [0.99531615 0.79591835 1.32288408 2.06342578]
Predict_pe1:
 [[0.15180631 0.21865726 0.0927571  0.09659772 0.14482462 0.22200137
  0.07335562]
 [0.17108716 0.1987277  0.09988903 0.0949066  0.13103363 0.219802
  0.08455379]
 [0.18820123 0.18907106 0.09493257 0.11014286 0.14446624 0.2035209
  0.06966513]
 [0.18717472 0.21710691 0.09491825 0.10193045 0.12091119 0.20405261
  0.07390592]
 [0.14363281 0.20362334 0.10798264 0.10834158 0.14157096 0.20557377
  0.0892749 ]
 [0.18946616 0.20553409 0.08942107 0.10370305 0.11950219 0.2222909
  0.07008245]
 [0.15765847 0.16680968 0.13910982 0.12840624 0.11270424 0.20899113
  0.08632042]]
Predict_pe2:
 [[0.1804389  0.22623526 0.09050377 0.09984207 0.13449012 0.20570919
  0.06278066]
 [0.17397057 0.23949078 0.09981484 0.08798774 0.09656271 0.2241881
  0.07798531]
 [0.1723618  0.22109677 0.09580866 0.10857145 0.13743667 0.19203794
  0.07268658]
 [0.19910878 0.238186   0.08170141 0.08761961 0.1

In [ ]:
np.save('/content/drive/My Drive/KDEF_and_AKDEF_R/NrmEfficientNet/Result_Predict/pe12_Data/Result.npy', result)
np.save('/content/drive/My Drive/KDEF_and_AKDEF_R/NrmEfficientNet/Result_Predict/pe12_Data/Predict_pe1.npy', predict_pe1)
np.save('/content/drive/My Drive/KDEF_and_AKDEF_R/NrmEfficientNet/Result_Predict/pe12_Data/Predict_pe2.npy', predict_pe2)